# Open AI मोडेलहरूलाई फाइन ट्यूनिङ

यो नोटबुक Open AI बाट प्रदान गरिएको [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) दस्तावेजमा आधारित छ।

फाइन-ट्यूनिङले तपाईंको अनुप्रयोगको लागि फाउन्डेशन मोडेलहरूको प्रदर्शन सुधार गर्दछ, यसलाई थप डेटा र सन्दर्भसँग पुन: प्रशिक्षण गरेर जुन त्यो विशेष प्रयोग केस वा परिदृश्यसँग सम्बन्धित हुन्छ। ध्यान दिनुहोस् कि प्रॉम्प्ट इन्जिनियरिङ प्रविधिहरू जस्तै _few shot learning_ र _retrieval augmented generation_ ले तपाईंलाई गुणस्तर सुधार गर्न सम्बन्धित डेटासँग डिफल्ट प्रॉम्प्टलाई सुधार गर्न अनुमति दिन्छ। यद्यपि, यी विधिहरू लक्षित फाउन्डेशन मोडेलको अधिकतम टोकन विन्डो साइजले सीमित छन्।

फाइन-ट्यूनिङसँग, हामी प्रभावकारी रूपमा मोडेललाई आवश्यक डेटासँग पुन: प्रशिक्षण गर्दैछौं (हामी अधिक उदाहरणहरू प्रयोग गर्न सक्छौं जुन अधिकतम टोकन विन्डोमा फिट हुन सक्दैन) - र एक _कस्टम_ संस्करण मोडेल तैनाथ गर्दैछौं जसलाई अब इन्फरेन्स समयमा उदाहरणहरू प्रदान गर्न आवश्यक पर्दैन। यसले मात्र हाम्रो प्रॉम्प्ट डिजाइनको प्रभावकारिता सुधार्दैन (हामी टोकन विन्डोलाई अन्य कुराहरूमा प्रयोग गर्न बढी लचिलोपन पाउँछौं) तर सम्भावित रूपमा हाम्रो लागतहरू पनि सुधार गर्दछ (इन्फरेन्स समयमा मोडेलमा पठाउनुपर्ने टोकनहरूको संख्या घटाएर)।

फाइन ट्यूनिङका ४ चरणहरू छन्:
1. प्रशिक्षण डेटा तयार पार्नुहोस् र अपलोड गर्नुहोस्।
1. फाइन-ट्यून गरिएको मोडेल प्राप्त गर्न प्रशिक्षण काम चलाउनुहोस्।
1. फाइन-ट्यून गरिएको मोडेलको मूल्याङ्कन गर्नुहोस् र गुणस्तरका लागि पुनरावृत्ति गर्नुहोस्।
1. सन्तुष्ट हुँदा फाइन-ट्यून गरिएको मोडेललाई इन्फरेन्सका लागि तैनाथ गर्नुहोस्।

ध्यान दिनुहोस् कि सबै फाउन्डेशन मोडेलहरूले फाइन-ट्यूनिङ समर्थन गर्दैनन् - नवीनतम जानकारीका लागि [OpenAI दस्तावेज जाँच गर्नुहोस्](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)। तपाईं पहिले फाइन-ट्यून गरिएको मोडेललाई पनि फाइन-ट्यून गर्न सक्नुहुन्छ। यस ट्युटोरियलमा, हामी `gpt-35-turbo` लाई हाम्रो लक्षित फाउन्डेशन मोडेलको रूपमा फाइन-ट्यूनिङका लागि प्रयोग गर्नेछौं।

---


### चरण 1.1: आफ्नो डेटासेट तयार पार्नुहोस्

हामी एउटा च्याटबोट बनाउनेछौं जसले तपाईंलाई तत्वहरूको आवधिक तालिका बुझ्न मद्दत गर्छ, एउटा तत्वको बारेमा प्रश्नहरूको जवाफ लिमरिकको रूपमा दिँदै। _यस_ सरल ट्युटोरियलमा, हामी मोडेललाई तालिम दिनको लागि केही नमूना प्रतिक्रियाहरूको अपेक्षित ढाँचा देखाउने डेटासेट मात्र सिर्जना गर्नेछौं। वास्तविक प्रयोगमा, तपाईंलाई धेरै बढी उदाहरणहरू सहितको डेटासेट बनाउन आवश्यक पर्छ। तपाईंले आफ्नो अनुप्रयोग डोमेनका लागि खुला डेटासेट (यदि उपलब्ध छ भने) पनि प्रयोग गर्न सक्नुहुन्छ र त्यसलाई फाइन-ट्युनिङका लागि पुनः ढाँचा बनाउन सक्नुहुन्छ।

हामी `gpt-35-turbo` मा केन्द्रित छौं र एकल-टर्न प्रतिक्रिया (च्याट कम्प्लीसन) खोज्दैछौं, त्यसैले हामी [यो सुझाइएको ढाँचा](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) प्रयोग गरेर उदाहरणहरू सिर्जना गर्न सक्छौं जुन OpenAI च्याट कम्प्लीसन आवश्यकताहरूलाई प्रतिबिम्बित गर्छ। यदि तपाईं बहु-टर्न संवादात्मक सामग्री अपेक्षा गर्नुहुन्छ भने, तपाईंले [बहु-टर्न उदाहरण ढाँचा](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) प्रयोग गर्नुहुनेछ जसमा `weight` प्यारामिटर हुन्छ जसले संकेत गर्छ कुन सन्देशहरू फाइन-ट्युनिङ प्रक्रियामा प्रयोग गरिनु पर्छ (वा हुँदैन)।

हामी यहाँ हाम्रो ट्युटोरियलका लागि सरल एकल-टर्न ढाँचा प्रयोग गर्नेछौं। डेटा [jsonl ढाँचामा](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) छ, प्रत्येक लाइनमा १ रेकर्ड हुन्छ, जुन JSON-फर्म्याट गरिएको वस्तुको रूपमा प्रतिनिधित्व गरिएको हुन्छ। तलको अंशले २ रेकर्डहरू नमूनाको रूपमा देखाउँछ - पूर्ण नमूना सेट (१० उदाहरणहरू) को लागि [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) हेर्नुहोस् जुन हामी हाम्रो फाइन-ट्युनिङ ट्युटोरियलमा प्रयोग गर्नेछौं। **नोट:** प्रत्येक रेकर्ड _अवश्य_ एकै लाइनमा परिभाषित हुनुपर्छ (सामान्य JSON फाइलमा जस्तो लाइनहरूमा विभाजित नगर्नुहोस्)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

वास्तविक प्रयोगमा राम्रो परिणामका लागि तपाईंलाई धेरै ठूलो उदाहरण सेट आवश्यक पर्छ - व्यापारिक सन्तुलन प्रतिक्रिया गुणस्तर र फाइन-ट्युनिङको समय/खर्च बीच हुनेछ। हामी सानो सेट प्रयोग गर्दैछौं ताकि प्रक्रिया देखाउन छिटो फाइन-ट्युनिङ पूरा गर्न सकियोस्। थप जटिल फाइन-ट्युनिङ ट्युटोरियलका लागि [यो OpenAI कुकबुक उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।


---

### चरण 1.2 तपाईंको डाटासेट अपलोड गर्नुहोस्

फाइलहरू API प्रयोग गरेर डाटा अपलोड गर्नुहोस् [यहाँ वर्णन गरिएको अनुसार](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)। ध्यान दिनुहोस् कि यो कोड चलाउनको लागि, तपाईंले पहिले निम्न चरणहरू पूरा गर्नुपर्छ:
 - `openai` Python प्याकेज स्थापना गर्नुभएको छ (नवीनतम सुविधाहरूको लागि संस्करण >=0.28.0 प्रयोग गर्न सुनिश्चित गर्नुहोस्)
 - `OPENAI_API_KEY` वातावरण चरलाई तपाईंको OpenAI API कुञ्जीमा सेट गर्नुभएको छ
थप जान्नको लागि, कोर्सका लागि प्रदान गरिएको [सेटअप गाइड](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) हेर्नुहोस्।

अब, तपाईंको स्थानीय JSONL फाइलबाट अपलोडको लागि फाइल सिर्जना गर्न कोड चलाउनुहोस्।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### चरण 2.1: SDK सँग फाइन-ट्यूनिङ काम सिर्जना गर्नुहोस्


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### चरण 2.2: कामको स्थिति जाँच गर्नुहोस्

यहाँ `client.fine_tuning.jobs` API सँग गर्न सकिने केही कुराहरू छन्:
- `client.fine_tuning.jobs.list(limit=<n>)` - अन्तिम n वटा फाइन-ट्यूनिङ कामहरूको सूची बनाउनुहोस्
- `client.fine_tuning.jobs.retrieve(<job_id>)` - कुनै विशेष फाइन-ट्यूनिङ कामको विवरण प्राप्त गर्नुहोस्
- `client.fine_tuning.jobs.cancel(<job_id>)` - फाइन-ट्यूनिङ काम रद्द गर्नुहोस्
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - कामबाट n सम्मका घटनाहरूको सूची बनाउनुहोस्
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

प्रक्रियाको पहिलो चरण _प्रशिक्षण फाइलको मान्यता_ हो ताकि डाटा सही ढाँचामा छ कि छैन सुनिश्चित गर्न सकियोस्।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### चरण 2.3: प्रगति अनुगमन गर्न घटनाहरू ट्र्याक गर्नुहोस्


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### चरण 2.4: OpenAI ड्यासबोर्डमा स्थिति हेर्नुहोस्


तपाईं OpenAI वेबसाइटमा गएर प्लेटफर्मको _Fine-tuning_ सेक्सन अन्वेषण गरेर पनि स्थिति हेर्न सक्नुहुन्छ। यसले तपाईंलाई वर्तमान कामको स्थिति देखाउनेछ, र साथै अघिल्लो कामको कार्यान्वयन रनहरूको इतिहास ट्र्याक गर्न पनि अनुमति दिनेछ। यस स्क्रिनशटमा, तपाईंले देख्न सक्नुहुन्छ कि अघिल्लो कार्यान्वयन असफल भयो, र दोस्रो रन सफल भयो। सन्दर्भका लागि, यो तब भयो जब पहिलो रनले गलत ढाँचामा रेकर्डहरू भएको JSON फाइल प्रयोग गर्यो - एक पटक सुधारिएपछि, दोस्रो रन सफलतापूर्वक पूरा भयो र मोडेल प्रयोगका लागि उपलब्ध गराइयो।

![Fine-tuning job status](../../../../../translated_images/ne/fine-tuned-model-status.563271727bf7bfba.webp)


तपाईंले दृश्य ड्यासबोर्डमा तल स्क्रोल गरेर स्थिति सन्देशहरू र मेट्रिक्सहरू पनि हेर्न सक्नुहुन्छ जस्तै देखाइएको छ:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/ne/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/ne/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Step 3.1: आईडी प्राप्त गर्नुहोस् र कोडमा फाइन-ट्यून गरिएको मोडेल परीक्षण गर्नुहोस्


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### चरण ३.२: प्लेग्राउन्डमा फाइन-ट्युन गरिएको मोडेल लोड र परीक्षण गर्नुहोस्

अब तपाईं दुई तरिकाले फाइन-ट्युन गरिएको मोडेल परीक्षण गर्न सक्नुहुन्छ। पहिलो, तपाईं प्लेग्राउन्डमा जान सक्नुहुन्छ र मोडेलहरू ड्रप-डाउनबाट तपाईंको नयाँ फाइन-ट्युन गरिएको मोडेल चयन गर्न सक्नुहुन्छ। अर्को विकल्प भनेको फाइन-ट्युनिङ प्यानलमा देखाइएको "प्लेग्राउन्ड" विकल्प प्रयोग गर्नु हो (माथिको स्क्रिनशट हेर्नुहोस्) जसले निम्न _तुलनात्मक_ दृश्य सुरु गर्छ जसले फाउन्डेसन र फाइन-ट्युन गरिएको मोडेल संस्करणहरू छिटो मूल्याङ्कनका लागि साइड-बाइ-साइड देखाउँछ।

![Fine-tuning job status](../../../../../translated_images/ne/fine-tuned-playground-compare.56e06f0ad8922016.webp)

सिर्फ तपाईंको प्रशिक्षण डाटामा प्रयोग गरिएको सिस्टम सन्दर्भ भर्नुहोस् र तपाईंको परीक्षण प्रश्न प्रदान गर्नुहोस्। तपाईंले देख्नुहुनेछ कि दुवै पक्षहरू समान सन्दर्भ र प्रश्नले अपडेट हुन्छन्। तुलना चलाउनुहोस् र तपाईंले तिनीहरू बीचको आउटपुटमा फरक देख्नुहुनेछ। _ध्यान दिनुहोस् कि फाइन-ट्युन गरिएको मोडेलले तपाईंले उदाहरणहरूमा दिएको ढाँचामा प्रतिक्रिया प्रस्तुत गर्छ भने फाउन्डेसन मोडेलले केवल सिस्टम प्रॉम्प्ट अनुसरण गर्छ_।

![Fine-tuning job status](../../../../../translated_images/ne/fine-tuned-playground-launch.5a26495c983c6350.webp)

तपाईंले देख्नुहुनेछ कि तुलना प्रत्येक मोडेलको टोकन गणना र अनुमान लगाउन लागेको समय पनि प्रदान गर्छ। **यो विशेष उदाहरण एक सरल प्रक्रिया देखाउनको लागि हो तर वास्तविक विश्वको डाटासेट वा परिदृश्यलाई प्रतिबिम्बित गर्दैन**। तपाईंले देख्न सक्नुहुन्छ कि दुवै नमूनाहरूमा समान संख्या टोकनहरू छन् (सिस्टम सन्दर्भ र प्रयोगकर्ता प्रॉम्प्ट समान छन्) र फाइन-ट्युन गरिएको मोडेलले अनुमान लगाउन बढी समय लिन्छ (कस्टम मोडेल)।

वास्तविक विश्वका परिदृश्यहरूमा, तपाईं यस प्रकारको खेलौना उदाहरण प्रयोग गर्नुहुने छैन, तर वास्तविक डाटाको विरुद्ध फाइन-ट्युनिङ गर्नुहुनेछ (जस्तै, ग्राहक सेवाका लागि उत्पादन सूची) जहाँ प्रतिक्रियाको गुणस्तर धेरै स्पष्ट हुनेछ। _त्यस_ सन्दर्भमा, फाउन्डेसन मोडेलसँग समान प्रतिक्रिया गुणस्तर प्राप्त गर्न थप कस्टम प्रॉम्प्ट इन्जिनियरिङ आवश्यक पर्छ जसले टोकन प्रयोग र सम्भावित रूपमा अनुमान लगाउन लागेको प्रक्रिया समय बढाउँछ। _यसलाई प्रयास गर्न, OpenAI कुकबुकमा फाइन-ट्युनिङका उदाहरणहरू हेर्नुहोस् र सुरु गर्नुहोस्।_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज यसको मूल भाषामा आधिकारिक स्रोत मानिनु पर्छ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
